# Using the `waterdata` module to pull data from the USGS Water Data APIs
The `waterdata` module will eventually replace the `nwis` module for accessing USGS water data. It leverages the [Water Data APIs](https://api.waterdata.usgs.gov/) to download metadata, daily values, and instantaneous values. 

While the specifics of this transition timeline are opaque, it is advised to switch to the new functions as soon as possible to reduce unexpected interruptions in your workflow.

As always, please report any issues you encounter on our [Issues](https://github.com/DOI-USGS/dataretrieval-python/issues) page. If you have questions or need help, please reach out to us at comptools@usgs.gov.

## Prerequisite: Get your Water Data API key
We highly suggest signing up for your own API key [here](https://api.waterdata.usgs.gov/signup/) to afford yourself higher rate limits and more reliable access to the data. If you opt not to register for an API key, then the number of requests you can make to the Water Data APIs is considerably lower, and if you share an IP address across users or workflows, you may hit those limits even faster. Luckily, registering for an API key is free and easy.

Once you've copied your API key and saved it in a safe place, you can set it as an environment variable in your python script for the current session:

```python
import os
os.environ['API_USGS_PAT'] = 'your_api_key_here'
``` 
Note that the environment variable name is `API_USGS_PAT`, which stands for "API USGS Personal Access Token".

If you'd like a more permanent, repository-specific solution, you can use the `python-dotenv` package to read your API key from a `.env` file in your repository root directory, like this:

```python
!pip install python-dotenv # only run this line once to install the package in your environment
from dotenv import load_dotenv
load_dotenv()  # this will load the environment variables from the .env file
```
Make sure your `.env` file contains the following line:
```
API_USGS_PAT=your_api_key_here
```
Also, do not commit your `.env` file to version control, as it contains sensitive information. You can add it to your `.gitignore` file to prevent accidental commits.

## Lay of the Land
Now that your API key is configured, it's time to take a 10,000-ft view of the functions in the `waterdata` module.

### Metadata endpoints
These functions retrieve metadata tables that can be used to refine your data requests.

- `get_reference_table()` - Not sure which parameter code you're looking for, or which hydrologic unit your study area is in? This function will help you find the right input values for the data endpoints to retrieve the information you want.
- `get_codes()` - Similar to `get_reference_table()`, this function retrieves dataframes containing available input values that correspond to the Samples water quality database.

### Data endpoints
- `get_daily()` - Daily values for monitoring locations, parameters, stat codes, and more.
- `get_continuous()` - Instantaneous values for monitoring locations, parameters, statistical codes, and more.
- `get_monitoring_locations()`- Monitoring location information such as name, monitoring location ID, latitude, longitude, huc code, site types, and more.
- `get_time_series_metadata()` - Timeseries metadata across monitoring locations, parameter codes, statistical codes, and more. Can be used to answer the question: what types of data are collected at my site(s) of interest and over what time period are/were they collected? 
- `get_latest_continuous()` - Latest instantaneous values for requested monitoring locations, parameter codes, statistical codes, and more.
- `get_latest_daily()` - Latest daily values for requested monitoring locations, parameter codes, statistical codes, and more.
- `get_field_measurements()` - Physically measured values (a.k.a discrete) of gage height, discharge, groundwater levels, and more for requested monitoring locations.
- `get_samples()` - Discrete water quality sample results for monitoring locations, observed properties, and more.

### A few key tips
- You'll notice that each of the data functions have many unique inputs you can specify. **DO NOT** specify too many! Specify *just enough* inputs to return what you need. But do not provide redundant geographical or parameter information as this may slow down your query and lead to errors.
- Each function returns a Tuple, containing a dataframe and a Metadata class. If you have `geopandas` installed in your environment, the dataframe will be a `GeoDataFrame` with a geometry included. If you do not have `geopandas`, the dataframe will be a `pandas` dataframe with the geometry contained in a coordinates column. The Metadata object contains information about your query, like the query url.
- If you do not want to return the `geometry` column, use the input `skip_geometry=True`.

## Examples
Let's get into some examples using the functions listed above. First, we need to load the `waterdata` module.

In [ ]:
import pandas as pd
from IPython.display import display
from datetime import datetime, timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
from dataretrieval import waterdata

In [ ]:
pcodes,metadata = waterdata.get_reference_table("parameter-codes")
display(pcodes.head())

Let's say we want to find all parameter codes relating to streamflow discharge. We can use some string matching to find applicable codes.

In [ ]:
streamflow_pcodes = pcodes[pcodes['parameter_name'].str.contains('streamflow|discharge', case=False, na=False)]
display(streamflow_pcodes[['parameter_code_id', 'parameter_name']])

Interesting that there are so many different streamflow-related parameter codes! Going on experience, let's use the most common one, `00060`, which is "Discharge, cubic feet per second".

Now that we know which parameter code we want to use, let's find all the stream monitoring locations that have recent discharge data and at least 10 years of daily values in the state of Nebraska. We will use the `waterdata.get_time_series_metadata()` function to suss out which sites fit the bill. This function will return a row for each *timeseries* that matches our inputs. It doesn't contain the daily discharge values themselves, just information *about* that timeseries.

First, let's get our expected date range in order. Note that the `waterdata` functions are capable of taking in bounded and unbounded date and datetime ranges. In this case, we want the start date of the discharge timeseries to be no more recent than 10 years ago, and we want the end date of the timeseries to be from at most a week ago.

In [ ]:
ten_years_ago =(date.today() - relativedelta(years=10))
one_week_ago = (datetime.now() - timedelta(days=7)).date()

In [ ]:
NE_discharge,_ = waterdata.get_time_series_metadata(
    state_name="Nebraska",
    parameter_code='00060',
    begin=f"1700-01-01/{ten_years_ago}",
    end=f"{one_week_ago}/..",
    skip_geometry=True
)

In [ ]:
display(NE_discharge.sort_values("monitoring_location_id").head())
print(f"There are {len(NE_discharge['monitoring_location_id'].unique())} sites with recent discharge data available in the state of Nebraska")

In the dataframe above, we are looking at 5 timeseries returned, ordered by monitoring location. You can also see that the first two rows show two different kinds of discharge for the same monitoring location: a mean daily discharge timeseries (with statistic id 00003, which represents "mean") and an instantaneous discharge timeseries (with statistic id 00011, which represents "points" or "instantaneous" values). Look closely and you may also notice that the `parent_timeseries_id` column for daily mean discharge matches the `time_series_id` for the instantaneous discharge. This is because once instantaneous measurements began at the site, they were used to calculate the daily mean.

Now that we know which sites have recent discharge data, let's find stream sites and plot them on a map. We will use the `waterdata.get_monitoring_locations()` function to grab more metadata about these sites. Even though we have a list of monitoring location IDs from the timeseries function, it's faster to use the `state_name` argument to return all stream sites, and then filter down to the ones we're interested in.

In [ ]:
NE_locations,_ = waterdata.get_monitoring_locations(
    state_name="Nebraska",
    site_type_code="ST"
    )

NE_locations_discharge = NE_locations.loc[NE_locations['monitoring_location_id'].isin(NE_discharge['monitoring_location_id'].unique().tolist())]
display(NE_locations_discharge[["monitoring_location_id", "monitoring_location_name", "hydrologic_unit_code"]].head())

If you have `geopandas` installed, the function will return a `GeoDataFrame` with a `geometry` column containing the monitoring locations' coordinates. If you don't have `geopandas` installed, it will return a regular `pandas` DataFrame with coordinate columns instead. Let's take a look at the site locations using `gpd.explore()`. Hover over the site points to see all the columns returned from `waterdata.get_monitoring_locations()`.

In [ ]:
NE_locations_discharge.set_crs(crs="WGS84").explore()

In [ ]:
ne_sites = NE_locations['monitoring_location_id'].to_list()
print(len(ne_sites))

We cannot feed 1700+ monitoring location ID's into the time series metadata function, so we will need to break this list up into smaller chunks and loop through them.

In [ ]:
chunk_size=50
chunks = [ne_sites[i:i + chunk_size] for i in range(0, len(ne_sites), chunk_size)]
len(chunks)

Now, we will loop through each chunk of sites and pull timeseries information for sites with discharge data from the past week and a timeseries that is at least 10 years old.

In [ ]:
dfs = pd.DataFrame()
for site_group in chunks:
        try:
            timeseries_info,_ = waterdata.get_time_series_metadata(
                monitoring_location_id=site_group,
                parameter_code='00060',
                begin=f"1700-01-01/{ten_years_ago}",
                end=f"{one_week_ago}/..",
                skip_geometry=True
            )
            if not timeseries_info.empty:
                dfs = pd.concat([dfs, timeseries_info])
        except Exception as e:
            # Log & continue; you can also implement retries here
            print(f"Batch failed (size={len(site_group)}): {e}")

display(dfs.head())

One thing you might notice is that this dataframe returns a `state_name` column!